# Densely Connected Convolutional Networks

## Introduction

If you like Neural Nets, you certainly have heard about VGGNet, Resnet, Inception Resnet, and others. These networks, each one in its time, reached state-of-the-art performance in some of the most famous challanges in Computer Vision we have. If we look at the short and successfully history of Neural Networks, pos the GPGPU revolution, we notice that year by year, these models got more powerful and more deeper. As a concequence, as networks were getting more and more dense, in the number of paramenters and layers, a new not so new problem emerged. How to prevent the gradient from vanishing by the time it reaches the first layers of the network?

To address this issue, many network architectures emerged such as the Resnet and Highway networks. Besides some changes, all of them tryed to solve this problem using one single approach. Create shortcut connections between layer so that the gradient signal could be propagated without much loss from the end to the begining of the netowrk.

In this context, arouse the Densely Connected Convolutional Networks, DenseNets. 

I have been using this architecute for a while for both, image classification and densely prediction problems such as Semantic Segmentation. During this time, I developed a library to use DenseNets using Tensorflow with its Slim package. I really like this model and in this post, we are going to do an overview of this archtecture, compare it with other very popular ones, and show how one might use the Library for its own pleasure. 

In [1]:
import tensorflow as tf
slim = tf.contrib.slim
from densenet import densenet_121
from densenet_utils import densenet_arg_scope

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
inputs = tf.placeholder(tf.float32, shape=[None, 784])
inputs_ = tf.reshape(inputs, [-1, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [4]:
with slim.arg_scope(densenet_arg_scope()):
    with slim.arg_scope(densenet_arg_scope()):
        logits, end_points = densenet_121(inputs_, 10, output_stride=8, is_training=True)

In [5]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
merged_summary_op = tf.summary.merge_all()

In [7]:
saver = tf.train.Saver()

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          inputs: batch[0], y_: batch[1]})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={inputs: batch[0], y_: batch[1]})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      inputs: mnist.test.images, y_: mnist.test.labels}))

step 0, training accuracy 0.08
step 100, training accuracy 0.14
step 200, training accuracy 0.26
step 300, training accuracy 0.24
step 400, training accuracy 0.26
step 500, training accuracy 0.34
step 600, training accuracy 0.46
step 700, training accuracy 0.5
step 800, training accuracy 0.54
step 900, training accuracy 0.68
step 1000, training accuracy 0.7
step 1100, training accuracy 0.74
step 1200, training accuracy 0.72
step 1300, training accuracy 0.84
step 1400, training accuracy 0.88
step 1500, training accuracy 0.86
step 1600, training accuracy 0.92
step 1700, training accuracy 0.86
step 1800, training accuracy 0.88
step 1900, training accuracy 0.94
step 2000, training accuracy 0.88
step 2100, training accuracy 0.96
step 2200, training accuracy 0.86
step 2300, training accuracy 0.96
step 2400, training accuracy 0.98
step 2500, training accuracy 0.92
step 2600, training accuracy 0.98
step 2700, training accuracy 0.94
step 2800, training accuracy 0.96
step 2900, training accuracy